In [ ]:
!git clone https://github.com/ninomiyalab/ML_MoQ

2版は1版から計算・メモリ効率を向上したもの

In [2]:
# from ML_MoQ.ML_QN import *
from ML_MoQ.ML_QN2 import *
# from ML_MoQ.ML_MoQ import *
from ML_MoQ.ML_MoQ2 import *

MNIST

In [ ]:
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense, Activation, Conv2D, Flatten
from tensorflow.keras import optimizers
import matplotlib.pyplot as plt
import numpy as np
import csv
import warnings

warnings.simplefilter('ignore')

def compare_MNIST(i = 0, verbose = False, draw_graph = True, draw_csv = False):
    # random seedを設定
    tf.random.set_seed(i)

    # サンプルの読み込み
    mnist = tf.keras.datasets.mnist
    (x_train, y_train), (x_test, y_test) = mnist.load_data()

    # サンプルの前処理
    x_train, x_test = x_train[..., np.newaxis]/255.0, x_test[..., np.newaxis]/255.0
    y_train, y_test = tf.keras.utils.to_categorical(y_train), tf.keras.utils.to_categorical(y_test)

    # モデルの定義
    def My_Model(input_shape, Output_dim):
        inputs = Input(shape = (input_shape))
        x = Flatten()(inputs)
        x = Dense(10, activation="sigmoid")(x)
        outputs = Dense(Output_dim, activation="softmax")(x)
        model = Model(inputs = [inputs], outputs = [outputs])
        return model
    
    # モデルの生成
    model = My_Model(x_train.shape[1:], Output_dim = y_train.shape[1:][0])
    # モデルの重みを保存
    model.save("model.h5")
    
    # 損失関数を指定
    loss_fn = tf.keras.losses.CategoricalCrossentropy()

    # エポックを指定
    epochs = 10000

    # CallBackを作成（train_accuracyに変化が見られなくなったら終了する）
    es_cb = keras.callbacks.EarlyStopping(monitor='accuracy', patience=15, verbose=0, mode='auto')

    # ML_QNの学習
    # --------------------------------------------------------------------------------------
    # モデルを読み込み
    model = load_model("model.h5")

    # Optimizerを指定
    # optimizer = ML_QN()
    optimizer = ML_QN2()
    
    # モデルをコンパイル
    model.compile(loss=loss_fn, optimizer=optimizer, metrics=['accuracy'])

    # 学習を行う
    ML_QN_history = model.fit(x_train, y_train, epochs = epochs, verbose = verbose, batch_size = x_train.shape[0], validation_data = (x_test, y_test), callbacks=[es_cb])
    # --------------------------------------------------------------------------------------

    # ML_MoQ(non-adaptive mu)の学習
    # --------------------------------------------------------------------------------------
    # モデルを読み込み
    model = load_model("model.h5")

    # Optimizerを指定
    # optimizer = ML_MoQ(adaptive_mu = False)
    optimizer = ML_MoQ2(adaptive_mu = False)
    
    # モデルをコンパイル
    model.compile(loss=loss_fn, optimizer=optimizer, metrics=['accuracy'])

    # 学習を行う
    ML_MoQ_history = model.fit(x_train, y_train, epochs = epochs, verbose = verbose, batch_size = x_train.shape[0], validation_data = (x_test, y_test), callbacks=[es_cb])
    # --------------------------------------------------------------------------------------

    # ML_MoQ(adaptive mu)の学習
    # --------------------------------------------------------------------------------------
    # モデルを読み込み
    model = load_model("model.h5")

    # Optimizerを指定
    # optimizer = ML_MoQ(adaptive_mu = True)
    optimizer = ML_MoQ2(adaptive_mu = True)
    
    # モデルをコンパイル
    model.compile(loss=loss_fn, optimizer=optimizer, metrics=['accuracy'])

    # 学習を行う
    adaptive_mu_ML_MoQ_history = model.fit(x_train, y_train, epochs = epochs, verbose = verbose, batch_size = x_train.shape[0], validation_data = (x_test, y_test), callbacks=[es_cb])
    # --------------------------------------------------------------------------------------

    # Adamの学習
    # --------------------------------------------------------------------------------------
    # モデルを読み込み
    model = load_model("model.h5")

    # Optimizerを指定
    optimizer = tf.keras.optimizers.Adam()
    
    # モデルをコンパイル
    model.compile(loss=loss_fn, optimizer=optimizer, metrics=['accuracy'])

    # 学習を行う
    Adam_history = model.fit(x_train, y_train, epochs = epochs, verbose = verbose, batch_size = x_train.shape[0], validation_data = (x_test, y_test), callbacks=[es_cb])
    # --------------------------------------------------------------------------------------
    # 結果のグラフの描画
    if draw_graph:
        # train_lossの描画
        fig = fig = plt.figure(figsize=(20, 10))
        ax = fig.add_subplot(111)
        fig.suptitle("train_loss")
        ax.plot(ML_QN_history.history['loss'],label="ML_QN")
        ax.plot(ML_MoQ_history.history['loss'],label="ML_MoQ")
        ax.plot(adaptive_mu_ML_MoQ_history.history['loss'],label="adaptive_mu_ML_MoQ")
        ax.plot(Adam_history.history['loss'],label="Adam")
        ax.set_xlabel('iterations')
        ax.set_ylabel('loss')
        ax.legend(bbox_to_anchor=(0, -0.2), loc='upper left', borderaxespad=0)
        ax.legend()
        plt.show()

        # train_accuracyの描画
        fig = fig = plt.figure(figsize=(20, 10))
        ax = fig.add_subplot(111)
        fig.suptitle("train_accuracy")
        ax.plot(ML_QN_history.history['accuracy'],label="ML_QN")
        ax.plot(ML_MoQ_history.history['accuracy'],label="ML_MoQ")
        ax.plot(adaptive_mu_ML_MoQ_history.history['accuracy'],label="adaptive_mu_ML_MoQ")
        ax.plot(Adam_history.history['accuracy'],label="Adam")
        ax.set_xlabel('iterations')
        ax.set_ylabel('accuracy')
        ax.legend(bbox_to_anchor=(0, -0.2), loc='upper left', borderaxespad=0)
        ax.legend()
        plt.show()

        # test_lossの描画
        fig = fig = plt.figure(figsize=(20, 10))
        ax = fig.add_subplot(111)
        fig.suptitle("test_loss")
        ax.plot(ML_QN_history.history['val_loss'],label="ML_QN")
        ax.plot(ML_MoQ_history.history['val_loss'],label="ML_MoQ")
        ax.plot(adaptive_mu_ML_MoQ_history.history['val_loss'],label="adaptive_mu_ML_MoQ")
        ax.plot(Adam_history.history['val_loss'],label="Adam")
        ax.set_xlabel('iterations')
        ax.set_ylabel('loss')
        ax.legend(bbox_to_anchor=(0, -0.2), loc='upper left', borderaxespad=0)
        ax.legend()
        plt.show()

        # test_accuracyの描画
        fig = fig = plt.figure(figsize=(20, 10))
        ax = fig.add_subplot(111)
        fig.suptitle("test_accuracy")
        ax.plot(ML_QN_history.history['val_accuracy'],label="ML_QN")
        ax.plot(ML_MoQ_history.history['val_accuracy'],label="ML_MoQ")
        ax.plot(adaptive_mu_ML_MoQ_history.history['val_accuracy'],label="adaptive_mu_ML_MoQ")
        ax.plot(Adam_history.history['val_accuracy'],label="Adam")
        ax.set_xlabel('iterations')
        ax.set_ylabel('accuracy')
        ax.legend(bbox_to_anchor=(0, -0.2), loc='upper left', borderaxespad=0)
        ax.legend()
        plt.show()

    # 結果をCSVに出力
    if draw_csv:
        with open("MNIST_result.csv", "a") as f:
            f.write(str(i))
            f.write("\ntrain_loss\n")
            f.write("ML_QN\n")
            f.write(",".join(list(map(str, ML_QN_history.history['loss']))))
            f.write("\nML_MoQ\n")
            f.write(",".join(list(map(str, ML_MoQ_history.history['loss']))))
            f.write("\nadaptive_mu_ML_MoQ\n")
            f.write(",".join(list(map(str, adaptive_mu_ML_MoQ_history.history['loss']))))
            f.write("\nAdam\n")
            f.write(",".join(list(map(str,Adam_history.history['loss']))))

            f.write("\ntrain_accuracy\n")
            f.write("ML_QN\n")
            f.write(",".join(list(map(str, ML_QN_history.history['accuracy']))))
            f.write("\nML_MoQ\n")
            f.write(",".join(list(map(str, ML_MoQ_history.history['accuracy']))))
            f.write("\nadaptive_mu_ML_MoQ\n")
            f.write(",".join(list(map(str, adaptive_mu_ML_MoQ_history.history['accuracy']))))
            f.write("\nAdam\n")
            f.write(",".join(list(map(str,Adam_history.history['accuracy']))))

            f.write("\ntest_loss\n")
            f.write("ML_QN\n")
            f.write(",".join(list(map(str, ML_QN_history.history['val_loss']))))
            f.write("\nML_MoQ\n")
            f.write(",".join(list(map(str, ML_MoQ_history.history['val_loss']))))
            f.write("\nadaptive_mu_ML_MoQ\n")
            f.write(",".join(list(map(str, adaptive_mu_ML_MoQ_history.history['val_loss']))))
            f.write("\nAdam\n")
            f.write(",".join(list(map(str,Adam_history.history['val_loss']))))

            f.write("\ntest_accuracy\n")
            f.write("ML_QN\n")
            f.write(",".join(list(map(str, ML_QN_history.history['val_accuracy']))))
            f.write("\nML_MoQ\n")
            f.write(",".join(list(map(str, ML_MoQ_history.history['val_accuracy']))))
            f.write("\nadaptive_mu_ML_MoQ\n")
            f.write(",".join(list(map(str, adaptive_mu_ML_MoQ_history.history['val_accuracy']))))
            f.write("\nAdam\n")
            f.write(",".join(list(map(str,Adam_history.history['val_accuracy']))))
            f.write("\n")
        
# nは実験の繰り返し回数
n = 10

# 学習の途中経過を出力するか
verbose = False

# 学習の結果のグラフを出力するか
draw_graph = True

# 学習の結果をCSVファイルに出力するか
draw_csv = False

for i in range(n):
    print(i + 1)
    compare_MNIST(i, verbose, draw_graph, draw_csv)